In [1]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()


True

In [2]:
!git clone https://github.com/laxmimerit/rag-dataset

Cloning into 'rag-dataset'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 43 (delta 10), reused 37 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 6.88 MiB | 5.85 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [12]:
from langchain_community.document_loaders import PyMuPDFLoader
import os

loader = PyMuPDFLoader(r"rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf")
loader.load()

[Document(metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf', 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:38:50+00:00', 'trapped': '', 'page': 0}, page_content='Citation: Espeño, P.R.; Ong, A.K.S.;\nGerman, J.D.; Gumasing, M.J.J.; Casas,\nE.S. Analysis of Actual Fitness\nSupplement Consumption among\nHealth and Fitness Enthusiasts. Foods\n2024, 13, 1424. https://doi.org/\n10.3390/foods13091424\nAcademic Editors: Ilija Djekic\nand Nada Smigic\nReceived: 30 March 2024\nRevised: 15 April 2024\nAccepted: 18 April 2024\nPublished: 6 May 2024\nCopyright: © 2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed\nunder\nthe\nterms\nand\nconditions of the Creative Commons\n

In [13]:
pdfs = []
for root, dirs, files in os.walk("rag-dataset"):
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

In [14]:
pdfs

['rag-dataset/finance/amazon/amazon-10-q-q3-2024.pdf',
 'rag-dataset/finance/google/goog-10-q-q3-2024.pdf',
 'rag-dataset/finance/facebook/Meta-09-30-2024-Exhibit-99-1_FINAL.pdf',
 'rag-dataset/finance/facebook/META-Q3-2024-Follow-Up-Call-Transcript.pdf',
 'rag-dataset/finance/facebook/Earnings-Presentation-Q3-2024.pdf',
 'rag-dataset/finance/facebook/META-Q3-2024-Earnings-Call-Transcript.pdf',
 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf',
 'rag-dataset/health supplements/3.health_supplements_side_effects.pdf',
 'rag-dataset/health supplements/2. Nutraceuticals research.pdf',
 'rag-dataset/gym supplements/2. High Prevalence of Supplement Intake.pdf',
 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf']

In [18]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

In [19]:
len(docs)

324

In [21]:
docs

[Document(metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2024-11-01T06:03:12-04:00', 'source': 'rag-dataset/finance/amazon/amazon-10-q-q3-2024.pdf', 'file_path': 'rag-dataset/finance/amazon/amazon-10-q-q3-2024.pdf', 'total_pages': 150, 'format': 'PDF 1.4', 'title': '0001018724-24-000161', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-Q filed on 2024-11-01 for the period ending 2024-09-30', 'keywords': '0001018724-24-000161; ; 10-Q', 'moddate': '2024-11-01T06:03:27-04:00', 'trapped': '', 'encryption': 'Standard V2 R3 128-bit RC4', 'page': 0}, page_content='Table of Contents\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n\xa0____________________________________\nFORM 10-Q\n____________________________________\xa0\n(Mark One)\n☒\nQUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended S

## Document Chunking

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=1000)
chunks = text_splitter.split_documents(docs)

In [24]:
len(docs), len(chunks)

(324, 8213)

In [25]:
docs[0].metadata
len(chunks[0].page_content)

947

In [26]:
chunks[150].metadata

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creator': 'EDGAR Filing HTML Converter',
 'creationdate': '2024-11-01T06:03:12-04:00',
 'source': 'rag-dataset/finance/amazon/amazon-10-q-q3-2024.pdf',
 'file_path': 'rag-dataset/finance/amazon/amazon-10-q-q3-2024.pdf',
 'total_pages': 150,
 'format': 'PDF 1.4',
 'title': '0001018724-24-000161',
 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group',
 'subject': 'Form 10-Q filed on 2024-11-01 for the period ending 2024-09-30',
 'keywords': '0001018724-24-000161; ; 10-Q',
 'moddate': '2024-11-01T06:03:27-04:00',
 'trapped': '',
 'encryption': 'Standard V2 R3 128-bit RC4',
 'page': 3}

## Document Vector Embedding

In [28]:
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [29]:
embeddings = OllamaEmbeddings(
    model='nomic-embed-text',
    base_url='http://localhost:11434'
)
vector = embeddings.embed_query(chunks[0].page_content)

In [30]:
len(vector)

768

## Storing Embedding in Vector Store

In [31]:
len(vector)
index = faiss.IndexFlatIP(len(vector))
index.ntotal, index.d

(0, 768)

In [32]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [33]:
vector_store.index.ntotal

0

In [34]:
ids = vector_store.add_documents(documents=chunks)

In [35]:
len(ids), vector_store.index.ntotal

(8213, 8213)

In [36]:
question = "how to gain muscle mass?"
result = vector_store.search(query=question, k=5, search_type='similarity')
result

[Document(id='f8a3d7e8-218e-4a2e-9f55-56f98454c7e8', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf', 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:38:50+00:00', 'trapped': '', 'page': 0}, page_content='individuals increase muscle mass, recover faster, and enhance their overall performance [2].\nWhey protein stands out as a top-quality protein source, containing more essential amino\nacids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for enduranc

In [37]:
db_name = "health_supplements"
vector_store.save_local(db_name)